#### 一、端午节的淘宝粽子交易

In [106]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'joyful-pandas-master\data\端午粽子数据.csv').convert_dtypes()
df.head()

,标题,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴
1,北京稻香村端午粽子手工豆沙粽220g*2袋散装豆沙粽香甜软糯豆沙粽,44,8人付款,天猫超市,上海
2,五芳斋粽子礼盒装鲜肉咸蛋黄大肉粽嘉兴豆沙甜粽端午团购散装礼品,89.9,100万+人付款,五芳斋官方旗舰店,浙江 嘉兴
3,稻香私房鲜肉粽蛋黄肉粽嘉兴粽子咸鸭蛋礼盒装端午节送礼特产团购,138,1936人付款,稻香村食品旗舰店,北京
4,嘉兴粽子 蛋黄鲜肉粽新鲜大肉粽早餐散装团购浙江特产蜜枣多口味,3.8,9500+人付款,城城喂食猫,浙江 嘉兴


In [107]:
df.columns

Index(['标题', ' 价格', '付款人数', '店铺', '发货地址 '], dtype='object')

(1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。

In [108]:
df1=df.drop(labels=df[df['发货地址 '].isna()].index,axis=0)
avg = df[df['发货地址 '].str.contains('上海')][' 价格'].astype("float").mean()
avg

64.86883306320908

(2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录？

In [109]:
df2 = df1[df1['标题'].str.contains('嘉兴')]
df2[~df2['发货地址 '].str.contains('嘉兴')].shape[0]

1032

(3) 请按照分位数将价格分为“高、较高、中、较低、低”5 个类别，再将
类别结果插入到标题一列之后，最后对类别列进行降序排序。

In [129]:
df_zongzi = df.copy().dropna(axis=0,subset=['发货地址 '])
df_zongzi.loc[:,' 价格'] = pd.to_numeric(df_zongzi.loc[:,' 价格'],errors='coerce')
q = [df_zongzi.loc[:,' 价格'].quantile(i) for i in [0,.2,.4,.6,.8,1]];q[-1] += 1
df_zongzi.loc[:,'价格类别'] = pd.cut(df_zongzi[' 价格'],q,right=False,labels=['低','较低','中','较高','高'])
ret_df = df_zongzi.loc[:,['标题','价格类别', ' 价格', '付款人数', '店铺', '发货地址 ']].sort_values(by='价格类别')
ret_df

,标题,价格类别,价格,付款人数,店铺,发货地址
3666,嘉兴五芳斋粽子紫米栗蓉粽板栗栗子甜粽子五谷素粽真空袋装美味,低,11.0,162人付款,三珍斋食品专营店,江苏 南京
1623,白米粽子新鲜湖南甜粽原味小粽子农家手工散装绿豆红枣碱水清水粽,低,20.9,181人付款,乐家_stone,湖南 常德
1053,【9只特价】新鲜蛋黄肉粽 嘉兴粽子特产端午节批发团购早餐粽手工,低,20.8,1358人付款,怡品三清旗舰店,浙江 杭州
3743,三珍斋嘉兴粽子大肉粽子蛋黄肉粽豆沙蜜枣端午节散装批发鲜肉棕子,低,19.9,35人付款,三珍斋乌镇专卖,浙江 嘉兴
1311,现做新鲜粽子咸蛋黄肉粽甜粽端午节散装嘉兴蜜枣豆沙大批發礼盒装,低,9.9,5000+人付款,徽老九美食,安徽 亳州
...,...,...,...,...,...,...
3548,锦华粽子瑞阳祥礼端午节礼盒新鲜嘉兴肉素六粒粽蛋卷组合送人礼品,高,110.6,36人付款,北京咏逢礼信商贸有限公司,北京
3523,印象昆韵新品昆曲文化创意端午节胚芽糙米手工肉粽子端阳礼品礼盒,高,156.0,21人付款,印象昆韵,江苏 苏州
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129.0,6人付款,五芳斋官方旗舰店,浙江 嘉兴
538,五芳斋粽子蛋黄鲜肉粽豆沙粽栗子肉棕子新鲜散装批发团购嘉兴肉粽,NaN,NaN,1.0万+人付款,五芳斋官方旗舰店,浙江 嘉兴


(4) 付款人数一栏有缺失值吗？若有则请利用上一问的分类结果对这些缺失
值进行合理估计并填充。

In [134]:
ret_df['人数数字'] = pd.to_numeric(ret_df['付款人数'].str.extract(r'([0-9.]*)')[0], errors='coerce')
include_10t = ret_df['付款人数'].str.contains(r'万')
ret_df['人数数字'][include_10t] = ret_df['人数数字'][include_10t] * 10000

for name, group in ret_df.groupby('价格类别'):
      ret_df.loc[group.index,'人数数字'] = group[[' 价格','人数数字']].sort_values(by=' 价格', ascending=False).interpolate()['人数数字']

ret_df['人数数字'] = ret_df['人数数字'].astype(int)
ret_df['付款人数'] = (ret_df['人数数字'].apply(str) + '人付款').to_frame()
ret_df.iloc[:,[0, 1, 2, 3, 4, 5]].head()

c:\users\chanceyin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,标题,价格类别,价格,付款人数,店铺,发货地址
3666,嘉兴五芳斋粽子紫米栗蓉粽板栗栗子甜粽子五谷素粽真空袋装美味,低,11.0,162人付款,三珍斋食品专营店,江苏 南京
1623,白米粽子新鲜湖南甜粽原味小粽子农家手工散装绿豆红枣碱水清水粽,低,20.9,181人付款,乐家_stone,湖南 常德
1053,【9只特价】新鲜蛋黄肉粽 嘉兴粽子特产端午节批发团购早餐粽手工,低,20.8,1358人付款,怡品三清旗舰店,浙江 杭州
3743,三珍斋嘉兴粽子大肉粽子蛋黄肉粽豆沙蜜枣端午节散装批发鲜肉棕子,低,19.9,35人付款,三珍斋乌镇专卖,浙江 嘉兴
1311,现做新鲜粽子咸蛋黄肉粽甜粽端午节散装嘉兴蜜枣豆沙大批發礼盒装,低,9.9,5000人付款,徽老九美食,安徽 亳州


(5) 请将数据后四列合并为如下格式的 Series：商品发货地为 ××，店铺为
××，共计 ×× 人付款，单价为 ××。

In [138]:
ret_df1 = ('商品发货地为' + ret_df['发货地址 '] + '，店铺为' + ret_df['店铺'] + \
        '，共计' + ret_df['人数数字'].apply(str) + '人付款，单价为' + ret_df[' 价格'].apply(str)).to_frame().rename(columns={0:'ID'})
ret_df1.head()

,ID
3666,商品发货地为江苏 南京，店铺为三珍斋食品专营店，共计162人付款，单价为11.0
1623,商品发货地为湖南 常德，店铺为乐家_stone，共计181人付款，单价为20.9
1053,商品发货地为浙江 杭州，店铺为怡品三清旗舰店，共计1358人付款，单价为20.8
3743,商品发货地为浙江 嘉兴，店铺为三珍斋乌镇专卖，共计35人付款，单价为19.9
1311,商品发货地为安徽 亳州，店铺为徽老九美食，共计5000人付款，单价为9.9


(6) 请将上一问中的结果恢复成原来的四列。

#### 二、墨尔本每日最低温度

(1) 剔除国庆节、五一劳动节和每月第一个周一，求每月的平均最低气温。

#### 三、2016 年 8 月上海市摩拜单车骑行记录

(1) 平均而言，周末单天用车量比工作日单天用车量更大吗？

In [102]:
df = pd.read_csv(r'joyful-pandas-master\data\摩拜单车数据.csv').convert_dtypes()
date_ext = df['start_time'].str.extract(r'([0-9]{4}-[0-9]{2}-[0-9]{2}) [0-9]{2}:[0-9]{2}')
df['Date'] = date_ext.astype('datetime64').rename(columns={0:'Date'})
df['Time'] = df['start_time'].astype('datetime64')
df.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,Date,Time
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",2016-08-20,2016-08-20 06:57:00
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",2016-08-29,2016-08-29 19:09:00
2,1106623,152045,8876,2016-08-13 16:17,121.383,31.254,2016-08-13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",2016-08-13,2016-08-13 16:17:00
3,1389484,196259,10648,2016-08-23 21:34,121.484,31.320,2016-08-23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",2016-08-23,2016-08-23 21:34:00
4,188537,78208,11735,2016-08-16 07:32,121.407,31.292,2016-08-16 07:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",2016-08-16,2016-08-16 07:32:00


(2) 工作日每天的高峰时间段大致为上午 7:30 至 9:30、下午 17:00 至 19:00，
请问 8 月里早高峰骑行记录量（以 start_time 为准）高于晚高峰的有
几天？

(3) 请给出在所有周五中（以 start_time 为准），记录条数最多的那个周五
所在的日期，并在该天内分别按 30 分钟、2 小时、6 小时统计摩拜单车
使用时间的均值。

(4) 请自行搜索相关代码或调用库，计算每条记录起点到终点的球面距离。

(5) 摩拜单车的骑行结束时间是以电子锁关闭的记录时间为准，但有时候用
户会忘记关锁，导致骑行时间出现异常。同时，正常人的骑行速度往往
大致落在一个合理的区间，请结合上一问中的球面距离和骑行起始、结
束时间，找出潜在的异常骑行记录。

(6) 由于路线的曲折性，起点到终点的球面距离往往不能充分反应行程长
度，请利用 track 列的路线坐标数据，计算估计实际骑行距离，并重新
仿照上一问的方法找出可能的异常记录。